In [ ]:
import os
import pickle
import numpy as np
from PIL import Image

def load_cifar10_meta(filename):
    with open(filename, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    return [label.decode('utf-8') for label in dict[b'label_names']]

def load_cifar10_batch(filename):
    with open(filename, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    images = dict[b'data']
    labels = dict[b'labels']
    images = images.reshape(len(images), 3, 32, 32).transpose(0, 2, 3, 1)
    return images, labels

def save_images(images, labels, class_names, path='images', prefix='image'):
    if not os.path.exists(path):
        os.makedirs(path)
    for i, (image, label) in enumerate(zip(images, labels)):
        class_name = class_names[label]
        class_path = os.path.join(path, class_name)
        if not os.path.exists(class_path):
            os.makedirs(class_path)
        img = Image.fromarray(image)
        img.save(os.path.join(class_path, f'{prefix}_{i}.png'))

def check_cifar10_folder(folder_path):
    batch_files = [os.path.join(folder_path, f'data_batch_{i}') for i in range(1, 6)]
    test_file = os.path.join(folder_path, 'test_batch')
    meta_file = os.path.join(folder_path, 'batches.meta')
    if all(os.path.exists(file) for file in batch_files) and os.path.exists(test_file) and os.path.exists(meta_file):
        return batch_files, test_file, meta_file
    else:
        return None, None, None

def process_cifar10_folder(cifar10_folder):
    batch_files, test_file, meta_file = check_cifar10_folder(cifar10_folder)

    if batch_files and test_file and meta_file:
        # Load class names
        class_names = load_cifar10_meta(meta_file)
        
        # Process training batches
        train_images = []
        train_labels = []
        for batch_file in batch_files:
            images, labels = load_cifar10_batch(batch_file)
            train_images.append(images)
            train_labels.append(labels)
        
        train_images = np.concatenate(train_images)
        train_labels = np.concatenate(train_labels)
        
        # Process test batch
        test_images, test_labels = load_cifar10_batch(test_file)
        
        # Determine the output paths
        images_output_path = os.path.join(cifar10_folder, 'images')
        train_output_path = os.path.join(images_output_path, 'train')
        test_output_path = os.path.join(images_output_path, 'test')
        
        # Save training images
        save_images(train_images, train_labels, class_names, path=train_output_path, prefix='train_image')
        
        # Save test images
        save_images(test_images, test_labels, class_names, path=test_output_path, prefix='test_image')
        
        # Print all labels
        print("Training labels:", train_labels)
        print("Test labels:", test_labels)
    else:
        print("CIFAR-10 files not found in the specified folder.")

# Path to the CIFAR-10 folder
cifar10_folder = r'C:\Users\lewka\Downloads\cifar-10-python\cifar-10-batches-py'

# Process the CIFAR-10 folder
process_cifar10_folder(cifar10_folder)